# Execute with tf tensorflow env

In [1]:
import numpy as np
import os
import tensorflow as tf


In [2]:
batch_size = 64
img_height = 172
img_width = 216

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  './Dataset',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,color_mode='grayscale')

Found 1710 files belonging to 3 classes.
Using 1368 files for training.


In [4]:
val_ds = tf.keras.utils.image_dataset_from_directory(
   './Dataset',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,color_mode='grayscale')

Found 1710 files belonging to 3 classes.
Using 342 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['1', '2', '3']


In [6]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(64, 172, 216, 1)
(64,)


In [7]:
#Change this if you want to normalise the images
normalization_layer = tf.keras.layers.Rescaling(1./255)


In [8]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.37254903


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
# Both models work well, the first is muuuuuch smaller
num_classes = 3

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])



model2 = tf.keras.models.Sequential([
    # 1st conv
  tf.keras.layers.Conv2D(96, (11,11),strides=(4,4), activation='relu', input_shape=( img_height, img_width,1)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2,2)),
    # 2nd conv
  tf.keras.layers.Conv2D(256, (11,11),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
     # 3rd conv
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 4th conv
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 5th Conv
  tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
  # To Flatten layer
  tf.keras.layers.Flatten(),
  # To FC layer 1
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  #To FC layer 2
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [11]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [12]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100
)

Epoch 1/100
22/22 [==============================] - 6s 57ms/step - loss: 0.7029 - accuracy: 0.7295 - val_loss: 0.4949 - val_accuracy: 0.8246
Epoch 2/100
22/22 [==============================] - 1s 30ms/step - loss: 0.5315 - accuracy: 0.8180 - val_loss: 0.4649 - val_accuracy: 0.8450
Epoch 3/100
22/22 [==============================] - 1s 30ms/step - loss: 0.4812 - accuracy: 0.8392 - val_loss: 0.4273 - val_accuracy: 0.8567
Epoch 4/100
22/22 [==============================] - 1s 30ms/step - loss: 0.4262 - accuracy: 0.8516 - val_loss: 0.4460 - val_accuracy: 0.8450
Epoch 5/100
22/22 [==============================] - 1s 29ms/step - loss: 0.4026 - accuracy: 0.8560 - val_loss: 0.4458 - val_accuracy: 0.8392
Epoch 6/100
22/22 [==============================] - 1s 30ms/step - loss: 0.3782 - accuracy: 0.8648 - val_loss: 0.3935 - val_accuracy: 0.8655
Epoch 7/100
22/22 [==============================] - 1s 31ms/step - loss: 0.3511 - accuracy: 0.8787 - val_loss: 0.3761 - val_accuracy: 0.8830
Epoch 

KeyboardInterrupt: 

In [13]:
model.save('model.h5')

In [ ]:
model3 =  tf.keras.models.load_model('model.h5')